In [1]:
from mpdaf.obj import Cube
import numpy as np
import pandas as pd

In [2]:
from dfisher_2022a.emission_lines import Halpha
from dfisher_2022a.io.cube import FitReadyCube, RestCube, SNRMap
from dfisher_2022a.io.line import Line
import dfisher_2022a as dfi

### Read in test data cube

In [3]:
path = "../dfisher_2022a/tests/data/"
fits_file = path + "single_gaussian_muse_size.fits"
res_file = path + "eso338_test_run_6563_simple_model.txt"

In [4]:
cube = Cube(fits_file)

In [5]:
cube.shape

(34, 368, 331)

In [6]:
cube.wave.get_crval()

6603.8486328125

### Settings

In [7]:
Z = 0.009482649107040553
SNR_THRESHOLD = 5

### Prepare data for fitting

#### 1. de-redshift

In [8]:
rcube = RestCube(cube, Z)

In [9]:
rcube.z

0.009482649107040553

In [10]:
rcube._restwave

6541.814897615204

In [11]:
rcube.restcube.wave.get_crval()

6541.814897615204

2. #### select fitting region

In [12]:
fline = Line(Halpha)

In [13]:
print(fline.low, fline.line, fline.high)

6547.819 6562.819 6577.819


In [14]:
rcube.restcube.wave.pixel(6577.8199)

28.804001907823775

In [15]:
frcube = FitReadyCube(rcube, fline)

In [16]:
frcube.low_index

5

In [17]:
frcube.cube.shape

(25, 368, 331)

#### 3. get snr map

In [18]:
snrmap = SNRMap(rcube.restcube, SNR_THRESHOLD)

In [19]:
snr = snrmap.snr

In [20]:
snr[80,80]

masked

In [21]:
smap = snrmap.map

In [22]:
smap.data[80,80]

masked

In [23]:
snr.shape

(368, 331)

#### get fit ready cube

In [24]:
ffrcube = FitReadyCube(rcube, fline, SNR_THRESHOLD)

In [25]:
ffrcube.cube.data[:,150,150]

masked_array(data=[12.175702095031738, 11.767293930053711,
                   12.22632884979248, 11.773338317871094,
                   11.761051177978516, 11.782549858093262,
                   12.78128719329834, 11.9537935256958,
                   14.307246208190918, 30.050148010253906,
                   87.67877960205078, 187.4704132080078,
                   241.05686950683594, 179.9756317138672,
                   81.16941833496094, 27.804880142211914,
                   14.032210350036621, 11.993968963623047,
                   11.892915725708008, 12.09134578704834,
                   11.779748916625977, 12.0618257522583,
                   11.765948295593262, 11.759703636169434,
                   12.781892776489258],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False],
       fill

### Fit the data

In [26]:
model = dfi.models.Lm_Const_1GaussModel

In [27]:
cube_for_fit = ffrcube.cube

In [31]:
small = cube_for_fit[:,140:150, 140:150]

#### Test with shared memory

In [29]:
c = cube_for_fit._data.reshape(25,368*331)

In [30]:
f = cube_for_fit._data

In [31]:
f.shape

(25, 368, 331)

In [32]:
f[:,3,7]

array([ 4.16298246,  4.20755577,  0.65867424,  2.57399011,  4.33883619,
        2.72161055, 10.93874931,  2.34243345,  1.76895237,  1.26190317,
        2.42319226,  0.81769162, 10.33891964,  2.76214838,  2.57704401,
        2.45183587,  1.8747642 ,  6.48021841,  0.26101619,  6.09778404,
       13.97373962,  1.92961216,  2.3155334 ,  0.33040407,  5.14313889])

In [33]:
c[:,1000]

array([ 4.16298246,  4.20755577,  0.65867424,  2.57399011,  4.33883619,
        2.72161055, 10.93874931,  2.34243345,  1.76895237,  1.26190317,
        2.42319226,  0.81769162, 10.33891964,  2.76214838,  2.57704401,
        2.45183587,  1.8747642 ,  6.48021841,  0.26101619,  6.09778404,
       13.97373962,  1.92961216,  2.3155334 ,  0.33040407,  5.14313889])

In [34]:
d = np.zeros((3,3))

In [35]:
d[:] = np.nan

In [36]:
d

array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]])

In [37]:
b = np.array([[[0,0,0],[1,1,1],[2,2,2]],[[3,3,3],[4,4,4],[5,5,5]]])

In [38]:
b.shape

(2, 3, 3)

In [39]:
b

array([[[0, 0, 0],
        [1, 1, 1],
        [2, 2, 2]],

       [[3, 3, 3],
        [4, 4, 4],
        [5, 5, 5]]])

In [40]:
c = b.reshape(2,9)

In [41]:
b[0,0,:]

array([0, 0, 0])

In [42]:
c[:,4]

array([1, 4])

In [34]:
datac = np.ctypeslib.as_ctypes(cube_for_fit.data)
global shared_data
shared_data = sharedctypes.RawArray(datac._type_, datac)

In [37]:
type(cube_for_fit.data)

numpy.ma.core.MaskedArray

In [38]:
ddata = np.ctypeslib.as_array(shared_data)

In [42]:
ddata[:,3,7]

array([ 4.16298246,  4.20755577,  0.65867424,  2.57399011,  4.33883619,
        2.72161055, 10.93874931,  2.34243345,  1.76895237,  1.26190317,
        2.42319226,  0.81769162, 10.33891964,  2.76214838,  2.57704401,
        2.45183587,  1.8747642 ,  6.48021841,  0.26101619,  6.09778404,
       13.97373962,  1.92961216,  2.3155334 ,  0.33040407,  5.14313889])

In [44]:
cdata = cube_for_fit.data.ctypes

In [52]:
type(cdata)

numpy.core._internal._ctypes

In [53]:
type(datac)

c_double_Array_331_Array_368_Array_25

In [55]:
import ctypes
# cdatac = np.ctypeslib.as_ctypes(cdata)
global shared_cdata
shared_cdata = sharedctypes.RawArray(cdata)

TypeError: RawArray() missing 1 required positional argument: 'size_or_initializer'

In [ ]:
rr = np.ctypeslib.as_array(shared_data)

In [ ]:
rr

In [ ]:
shared_data

In [ ]:
cube_for_fit.data.__array_interface__

In [33]:
from multiprocessing import Pool, RawArray, sharedctypes, Process

In [46]:
type(small.data)

numpy.ma.core.MaskedArray

In [32]:
fc = dfi.fits.base.FitCube(small, model)

In [34]:
fc.res.shape

(400, 1)

In [31]:
fc.data.shape

(25, 10, 10)

In [37]:
%%timeit
c = fc.fit_single_spaxel(0,1)

subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subprocess:  4732  pixel:  0 1
subproce

In [33]:
%%time

fc.fit_all(4,20)

subprocess:  4931  pixel:  20
subprocess: subprocess:   4930 4932 pixel:   subprocess:  4933  pixel:  60
0subprocess:  4933  pixel: start pooling
 61

subprocess:  4930  pixel:  pixel: 
 40subprocess:  4931 subprocess:   pixel: 4932   pixel:  2141

 1
subprocess:  4933  pixel:  62
subprocess: subprocess:   4931 4930 pixel:    pixel: 22 2

subprocess:  4932  pixel:  42
subprocess:  4933  pixel:  63
subprocess: subprocess:  4930   pixel: 4931  3
subprocess:  4932  pixel:  43
 pixel:  23
subprocess:  4933  pixel:  64subprocess: 
 4930  pixel:  4
subprocess:  4931  pixel:  24
subprocess:  4932  pixel:  44
subprocess:  4933  pixel:  65
subprocess: subprocess: subprocess:  4932  pixel:  45
 4930  pixel:  5
 4931  pixel:  25
subprocess:  4933  pixel:  66
subprocess:  4930  pixel:  6
subprocess:  4931  pixel:  26subprocess:  4932  pixel:  46
subprocess:  4932  pixel:  47

subprocess:  4930subprocess:   pixel:   7
4933  pixel:  67subprocess: 
 4930  pixel:  8
subprocess:  4933  pixel:  68
subpr

In [37]:
fc.records.sum()

109.75588798522949

In [32]:
np.zeros(3)

array([0., 0., 0.])

In [40]:
tt = fc.res["Result"][0]

In [42]:
%%time
fc.extract_info()

CPU times: user 5.23 s, sys: 535 ms, total: 5.77 s
Wall time: 14.3 s


In [37]:
df = fc.res

In [38]:
type(df)

pandas.core.frame.DataFrame

In [41]:
tt = df.drop(axis=1, columns="Result")

In [53]:
tt.to_csv("fitting_result.txt", index=False, sep='\t', float_format='%.5f')

In [37]:
class Wec():
    def __init__(self):
        pass
    def add_name(self,name):
        self.name = name

In [51]:
vars(Wec().add_name('tt'))

TypeError: vars() argument must have __dict__ attribute

In [45]:
a.add_name('tt')

In [46]:
getattr(a, 'name')

'tt'

In [37]:
fsmall = dfi.fits.base.FitCube(small, model)

/Users/SuperTiger/ADACS/repo/SS2022A-DFisher/dfisher_2022a/dfisher_2022a/fits/base.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["row"][i*axis_y:(i+1)*axis_y] = i


In [38]:
ch = fc.res["Result"][0]

In [55]:
type(ch)

lmfit.model.ModelResult

In [56]:
dir(ch)

['_Minimizer__jacobian',
 '_Minimizer__residual',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abort',
 '_calculate_covariance_matrix',
 '_calculate_uncertainties_correlations',
 '_err_max_evals',
 '_err_nonparam',
 '_int2ext_cov_x',
 '_lnprob',
 '_repr_html_',
 'aborted',
 'aic',
 'ampgo',
 'basinhopping',
 'best_fit',
 'best_values',
 'bic',
 'brute',
 'calc_covar',
 'call_kws',
 'chisqr',
 'ci_out',
 'ci_report',
 'col_deriv',
 'components',
 'conf_interval',
 'covar',
 'data',
 'dual_annealing',
 'dump',
 'dumps',
 'emcee',
 'errorbars',
 'eval',
 'eval_components',
 'eval_uncertainty',
 'fit',
 'fit_report',
 'flatchain',
 'ier',
 'init_fit',
 'init_params',


In [39]:
df = fc.res

In [71]:
def get_chi(x):
    return pd.Series(x.chisqr)

In [73]:
df["chi"] = fc.res["Result"].apply(get_chi)

In [74]:
df

,row,col,Result,chi
0,0,0,<lmfit.model.ModelResult object at 0x14a239880>,1.641783
1,0,1,<lmfit.model.ModelResult object at 0x13141c580>,0.425958
2,0,2,<lmfit.model.ModelResult object at 0x1314666a0>,1.536552
3,0,3,<lmfit.model.ModelResult object at 0x14a22a130>,2.285237
4,0,4,<lmfit.model.ModelResult object at 0x14a26c640>,0.318398
...,...,...,...,...
95,9,5,<lmfit.model.ModelResult object at 0x14a26c610>,0.559235
96,9,6,<lmfit.model.ModelResult object at 0x14a265cd0>,0.584941
97,9,7,<lmfit.model.ModelResult object at 0x14a26c430>,1.032410
98,9,8,<lmfit.model.ModelResult object at 0x14a26c040>,0.163493


In [58]:
cr = ch.result

In [63]:
cr.var_names

['g1_height', 'g1_center', 'g1_sigma', 'c']

In [60]:
g = ch.params.get('g1_height')

In [63]:
g.stderr

1.0535887534869381

In [64]:
g

<Parameter 'g1_height', value=137.97185427421581 +/- 1.05, bounds=[0:inf]>

In [56]:
ch.last_internal_values

array([1.38968256e+02, 6.56301940e+03, 2.21056375e+00, 5.27618346e+00])

In [58]:
ch.best_values

{'c': 5.2761834589929615,
 'g1_height': 137.97185427421581,
 'g1_center': 6563.019402970259,
 'g1_sigma': 1.4262300169236233}

In [85]:
type(ch)

lmfit.model.ModelResult

In [86]:
import sys

In [87]:
sys.getsizeof(ch)

48

In [88]:
id(ch)

5220214144

In [91]:
l = [ch,ch,ch]

In [94]:
sys.getsizeof(l)

86456

In [93]:
l = [ch]*10800

In [96]:
y = (1,2,'tr')

In [98]:
type(y)

tuple

In [40]:
def f(x):
    return x + 1

In [43]:
for ele in map(f, list(range(10))):
    print(ele)

1
2
3
4
5
6
7
8
9
10


In [44]:
gg = map(f, list(range(10)))

In [45]:
type(gg)

map

In [50]:
gg.__next__()

3

In [51]:
def f(q):
    q.put([42, None, 'hello'])

In [143]:
from multiprocessing import Queue, Process, Pool, Manager, RawArray, Array

In [53]:
q = Queue()

In [55]:
f(q)

In [59]:
q.get()

[42, None, 'hello']

In [61]:
p = Process()

In [189]:
p.start()

In [190]:
p.pid

2840

In [65]:
p.__dict__

{'_identity': (71,),
 '_config': {'authkey': b'\x04\xe2\t\xb2f_?\xb110\x01rC\x04\xc7\xbf\xb3\xf2\xd9\x0c/\x1d\xdcR8\xbb\xf3\x0f\xe3\xf2\xf3\x92',
  'semprefix': '/mp'},
 '_parent_pid': 82516,
 '_parent_name': 'MainProcess',
 '_popen': None,
 '_closed': False,
 '_target': None,
 '_args': (),
 '_kwargs': {},
 '_name': 'Process-71'}

In [66]:
p2 = Process()

In [67]:
p2.__dict__

{'_identity': (72,),
 '_config': {'authkey': b'\x04\xe2\t\xb2f_?\xb110\x01rC\x04\xc7\xbf\xb3\xf2\xd9\x0c/\x1d\xdcR8\xbb\xf3\x0f\xe3\xf2\xf3\x92',
  'semprefix': '/mp'},
 '_parent_pid': 82516,
 '_parent_name': 'MainProcess',
 '_popen': None,
 '_closed': False,
 '_target': None,
 '_args': (),
 '_kwargs': {},
 '_name': 'Process-72'}

In [191]:
pool = Pool()

In [195]:
for p in pool._pool:
    print(p)
    

<SpawnProcess name='SpawnPoolWorker-187' pid=3171 parent=82516 started daemon>
<SpawnProcess name='SpawnPoolWorker-188' pid=3172 parent=82516 started daemon>
<SpawnProcess name='SpawnPoolWorker-189' pid=3173 parent=82516 started daemon>
<SpawnProcess name='SpawnPoolWorker-190' pid=3174 parent=82516 started daemon>


In [192]:
pool.__dict__

{'_pool': [<SpawnProcess name='SpawnPoolWorker-187' pid=3171 parent=82516 started daemon>,
  <SpawnProcess name='SpawnPoolWorker-188' pid=3172 parent=82516 started daemon>,
  <SpawnProcess name='SpawnPoolWorker-189' pid=3173 parent=82516 started daemon>,
  <SpawnProcess name='SpawnPoolWorker-190' pid=3174 parent=82516 started daemon>],
 '_state': 'RUN',
 '_ctx': <multiprocessing.context.SpawnContext at 0x11acd9f40>,
 '_inqueue': <multiprocessing.queues.SimpleQueue at 0x14fbc8580>,
 '_outqueue': <multiprocessing.queues.SimpleQueue at 0x14fbc8a90>,
 '_quick_put': <bound method _ConnectionBase.send of <multiprocessing.connection.Connection object at 0x14fbc8dc0>>,
 '_quick_get': <bound method _ConnectionBase.recv of <multiprocessing.connection.Connection object at 0x14fbc8400>>,
 '_taskqueue': <_queue.SimpleQueue at 0x14fbd6360>,
 '_change_notifier': <multiprocessing.queues.SimpleQueue at 0x14fb808e0>,
 '_cache': {},
 '_maxtasksperchild': None,
 '_initializer': None,
 '_initargs': (),
 '_

In [76]:
m = Manager()

In [78]:
dir(m)

['Array',
 'Barrier',
 'BoundedSemaphore',
 'Condition',
 'Event',
 'JoinableQueue',
 'Lock',
 'Namespace',
 'Pool',
 'Queue',
 'RLock',
 'Semaphore',
 'Value',
 '_Client',
 '_Listener',
 '_Server',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_address',
 '_authkey',
 '_create',
 '_ctx',
 '_debug_info',
 '_finalize_manager',
 '_number_of_objects',
 '_process',
 '_registry',
 '_run_server',
 '_serializer',
 '_state',
 'address',
 'connect',
 'dict',
 'get_server',
 'join',
 'list',
 'register',
 'shutdown',
 'start']

In [74]:
p2.pid

99365

In [69]:
pool.__dict__

{'_pool': [<SpawnProcess name='SpawnPoolWorker-73' pid=99353 parent=82516 started daemon>,
  <SpawnProcess name='SpawnPoolWorker-74' pid=99354 parent=82516 started daemon>,
  <SpawnProcess name='SpawnPoolWorker-75' pid=99355 parent=82516 started daemon>,
  <SpawnProcess name='SpawnPoolWorker-76' pid=99356 parent=82516 started daemon>],
 '_state': 'RUN',
 '_ctx': <multiprocessing.context.SpawnContext at 0x11acd9f40>,
 '_inqueue': <multiprocessing.queues.SimpleQueue at 0x14e49fa30>,
 '_outqueue': <multiprocessing.queues.SimpleQueue at 0x14e49ff40>,
 '_quick_put': <bound method _ConnectionBase.send of <multiprocessing.connection.Connection object at 0x14e49fb20>>,
 '_quick_get': <bound method _ConnectionBase.recv of <multiprocessing.connection.Connection object at 0x14e49f4f0>>,
 '_taskqueue': <_queue.SimpleQueue at 0x14eb52950>,
 '_change_notifier': <multiprocessing.queues.SimpleQueue at 0x14e309b50>,
 '_cache': {},
 '_maxtasksperchild': None,
 '_initializer': None,
 '_initargs': (),
 '_

In [79]:
m = Manager()

In [81]:
temp_dict = m.dict()

In [83]:
temp_dict.__dict__

{'_tls': <multiprocessing.util.ForkAwareLocal at 0x14ebadb80>,
 '_idset': {'10e23cd40'},
 '_token': Token(typeid='dict', address='/var/folders/nb/wym_qb_d4pxc93dj_c7l3wlr0000gn/T/pymp-qed3y3vf/listener-w2rmtmkq', id='10e23cd40'),
 '_id': '10e23cd40',
 '_manager': <multiprocessing.managers.SyncManager at 0x14e3e7250>,
 '_serializer': 'pickle',
 '_Client': <function multiprocessing.connection.Client(address, family=None, authkey=None)>,
 '_owned_by_manager': False,
 '_authkey': b'\x04\xe2\t\xb2f_?\xb110\x01rC\x04\xc7\xbf\xb3\xf2\xd9\x0c/\x1d\xdcR8\xbb\xf3\x0f\xe3\xf2\xf3\x92',
 '_close': <Finalize object, callback=_decref, args=(Token(typeid='dict', address='/var/folders/nb/wym_qb_d4pxc93dj_c7l3wlr0000gn/T/pymp-qed3y3vf/listener-w2rmtmkq', id='10e23cd40'), b'\x04\xe2\t\xb2f_?\xb110\x01rC\x04\xc7\xbf\xb3\xf2\xd9\x0c/\x1d\xdcR8\xbb\xf3\x0f\xe3\xf2\xf3\x92', <multiprocessing.managers.State object at 0x14e3e7bb0>, <multiprocessing.util.ForkAwareLocal object at 0x14ebadb80>, ProcessLocalSet({

In [84]:
def test(idx, test_dict):
    test_dict[idx] = idx

In [85]:
pool = Pool()

In [89]:
c = np.zeros((4,4))

In [169]:
ar = m.Array('d',range(100))

In [175]:
ar

<ArrayProxy object, typeid 'Array' at 0x1357f0bb0>

In [180]:
ar._getvalue()

array('d', [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0])

In [129]:
x_shape = (10,10)

In [182]:
data = np.random.rand(*(10,10))

In [131]:
data.shape

(10, 10)

In [159]:
x_np = np.frombuffer(ar.get_obj(), dtype=np.float64)

In [160]:
x_np

array([10., 10.])

In [152]:
dir(x_np)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',

In [198]:
x_np

array([10., 10.])

In [135]:
type(ar)

multiprocessing.sharedctypes.c_double_Array_100

In [141]:
m.Value('d',1)

<ValueProxy object, typeid 'Value' at 0x14f969910>

In [144]:
arr = Array('d',100)

In [145]:
type(arr)

multiprocessing.sharedctypes.SynchronizedArray

In [184]:
import os

In [187]:
os.getpid()

82516

In [196]:
arr.base

AttributeError: 'SynchronizedArray' object has no attribute 'base'

In [220]:
a = [2.0, 3,4]

In [221]:
id(a)

5626133312

In [222]:
b = a

In [223]:
id(b)

5626133312

In [225]:
hex(5626133312)

'0x14f57fb40'

In [226]:
aa = a[0]

In [229]:
id(aa)

5632158672

In [235]:
bb = a[0]

In [236]:
id(bb)

5632158672

In [210]:
b.__hash__

<method-wrapper '__hash__' of float object at 0x14eaa72d0>

In [230]:
p = Process()

In [231]:
p.start()

In [232]:
p.pid

3862

In [233]:
c = a[0]

In [234]:
id(c)

5632158672

In [237]:
a[0] = c + 1

In [238]:
p.join()

In [239]:
c

2.0

In [240]:
a

[3.0, 3, 4]